## dependencies


In [21]:
!apt-get install -y tesseract-ocr
!pip install pytesseract pillow pdf2image
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


##ocr

In [22]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os

def ocr_image(path):
    img = Image.open(path)
    return pytesseract.image_to_string(img)

def ocr_pdf(path):
    pages = convert_from_path(path)
    text = ""
    for i, page in enumerate(pages):
        text += f"\n--- Page {i+1} ---\n"
        text += pytesseract.image_to_string(page)
    return text

file_path = "/content/sample-report.pdf"  # change file name

if file_path.lower().endswith(".pdf"):
    extracted_text = ocr_pdf(file_path)
else:
    extracted_text = ocr_image(file_path)

with open("ocr_output.txt", "w", encoding="utf-8") as f:
    f.write(extracted_text)

print("OCR done")


OCR done


##SLA and Clause extraction

In [23]:
import re

# ------------------ Helpers ------------------

def clean_text(text):
    # basic normalization to reduce OCR noise
    t = text.replace('\r', ' ').replace('\n', ' ')
    t = re.sub(r'\s+', ' ', t)         # collapse whitespace
    t = t.replace('O', '0')            # optional: common OCR mistake (careful)
    return t.strip()

def parse_amount(s):
    if not s: return None
    s = s.replace(',', '').replace(' ', '')
    # handle $ and parentheses etc
    s = s.replace('(', '').replace(')', '')
    m = re.search(r'(-)?\s*\$?([0-9]*\.?[0-9]+)([kK])?', s)
    if not m:
        # try to extract digits any way
        m2 = re.search(r'([0-9]*\.?[0-9]+)([kK])?', s)
        if not m2: return None
        sign, num, k = None, m2.group(1), m2.group(2)
    else:
        sign, num, k = m.group(1), m.group(2), m.group(3)
    val = float(num)
    if k: val *= 1000
    if sign: val = -val
    return val

def parse_percentage(s):
    if not s: return None
    m = re.search(r'([0-9]+(?:\.[0-9]+)?)', s)
    return float(m.group(1)) if m else None

def parse_int(s):
    if not s: return None
    s = s.replace(',', '').strip()
    m = re.search(r'([0-9]+)', s)
    return int(m.group(1)) if m else None

def find_first(patterns, text):
    """Try a list of patterns; return first match group(1) or None"""
    for p in patterns:
        m = re.search(p, text, re.IGNORECASE)
        if m:
            # prefer group 1 if exists; otherwise full match
            return m.group(1) if m.lastindex else m.group(0)
    return None

# ------------------ Field extractors ------------------

def extract_apr(text):
    patterns = [
        r'(?:APR|annual percentage rate|interest rate)[^\d%]{0,15}([0-9]+(?:\.[0-9]+)?\s*%)',
        r'interest rate[^\d]{0,15}([0-9]+(?:\.[0-9]+)?)\s*percent'
    ]
    raw = find_first(patterns, text)
    return raw, parse_percentage(raw) if raw else (None, None)

def extract_lease_term(text):
    patterns = [
        r'(?:lease term|term of lease|term)[:\s]{0,10}([0-9]{1,3}\s*(?:months|mos|yrs|years|year))',
        r'([0-9]{1,3}\s*(?:months|mos))',
        r'([0-9]{1,2})\s*(?:years|yrs)'
    ]
    raw = find_first(patterns, text)
    if not raw:
        return None, None
    # normalize to months if possible
    m = re.search(r'([0-9]{1,3})\s*(months|mos)', raw, re.IGNORECASE)
    if m:
        return raw, int(m.group(1))
    m2 = re.search(r'([0-9]{1,2})\s*(years|yrs|year)', raw, re.IGNORECASE)
    if m2:
        return raw, int(m2.group(1)) * 12
    # fallback: numeric only
    n = parse_int(raw)
    return raw, n

def extract_monthly_payment(text):
    # look for forms: "monthly payment $420", "payment of $420 per month", "installment $420"
    patterns = [
        r'(?:monthly payment|monthly instalment|monthly installment|monthly lease payment)[^\d\$]{0,20}(\$?[0-9,]+(?:\.[0-9]+)?\s*[kK]?)',
        r'([Pp]ayment of|installment of|installment)[^\d\$]{0,20}(\$?[0-9,]+(?:\.[0-9]+)?\s*[kK]?)\s*(?:per month|/month|monthly)?',
        r'(\$[0-9,]+(?:\.[0-9]+)?\s*(?:per month|/month|monthly))'
    ]
    # patterns return group 1 or 2; unify by searching manually
    for p in patterns:
        m = re.search(p, text, re.IGNORECASE)
        if m:
            # pick last capturing group with digits
            for g in reversed(m.groups()):
                if g and re.search(r'[0-9]', g):
                    raw = g
                    return raw, parse_amount(raw)
    return None, None

def extract_down_payment(text):
    patterns = [
        r'down payment[^\d\$]{0,20}(\$?[0-9,]+(?:\.[0-9]+)?\s*[kK]?)',
        r'deposit[^\d\$]{0,20}(\$?[0-9,]+(?:\.[0-9]+)?\s*[kK]?)'
    ]
    raw = find_first(patterns, text)
    return raw, parse_amount(raw) if raw else (None, None)

def extract_residual_value(text):
    patterns = [
        r'(?:residual value|residual)[^\d\$]{0,20}(\$?[0-9,]+(?:\.[0-9]+)?\s*[kK]?)',
        r'residual amount[^\d\$]{0,20}(\$?[0-9,]+)'
    ]
    raw = find_first(patterns, text)
    return raw, parse_amount(raw) if raw else (None, None)

def extract_mileage_and_overage(text):
    # mileage allowance per year
    mileage_patterns = [
        r'(\d{1,3}(?:,[0-9]{3})?|\d{1,2}k)\s*(?:miles per year|miles/year|mi/year|miles per annum|miles p[er]{0,2} year)',
        r'mileage allowance[^\d]{0,20}(\d{1,3}(?:,[0-9]{3})?|\d{1,2}k)'
    ]
    overage_patterns = [
        r'(\$[0-9]+(?:\.[0-9]+)?)\s*(?:per mile|/mile|per mi)',
        r'(?:overage|excess mileage|excess miles)[^\d\$]{0,20}(\$?[0-9]+(?:\.[0-9]+)?)'
    ]
    raw_m = find_first(mileage_patterns, text)
    raw_o = find_first(overage_patterns, text)
    # normalize mileage ('12k' -> 12000)
    def norm_mileage(s):
        if not s: return None
        s = s.lower().replace(',', '').strip()
        k = False
        if s.endswith('k'):
            k = True
            s = s[:-1]
        try:
            val = int(float(s))
            if k: val *= 1000
            return val
        except:
            return None
    return raw_m, norm_mileage(raw_m), raw_o, parse_amount(raw_o) if raw_o else (None, None)

def extract_buyout_price(text):
    patterns = [
        r'(?:buyout|purchase option|purchase price|purchase option price)[^\d\$]{0,30}(\$?[0-9,]+(?:\.[0-9]+)?\s*[kK]?)',
        r'payoff amount[^\d\$]{0,30}(\$?[0-9,]+)'
    ]
    raw = find_first(patterns, text)
    return raw, parse_amount(raw) if raw else (None, None)

# ------------------ Clause snippet extraction ------------------

def extract_clause_snippet(text, keywords, window=300):
    # return first found snippet around any of the keywords
    tiny = text.lower()
    for kw in keywords:
        i = tiny.find(kw.lower())
        if i != -1:
            start = max(0, i - 60)
            end = min(len(text), i + window)
            return text[start:end].strip()
    return None

# ------------------ Main wiring ------------------

if __name__ == "__main__":
    with open("ocr_output.txt", "r", encoding="utf-8") as fh:
        raw_text = fh.read()

    text = clean_text(raw_text)

    result = {}

    # APR
    raw_apr, apr_val = extract_apr(text)
    result['apr_raw'] = raw_apr
    result['apr_percent'] = apr_val

    # Lease term
    raw_term, term_months = extract_lease_term(text)
    result['lease_term_raw'] = raw_term
    result['lease_term_months'] = term_months

    # Monthly payment
    raw_monthly, monthly_val = extract_monthly_payment(text)
    result['monthly_payment_raw'] = raw_monthly
    result['monthly_payment'] = monthly_val

    # Down payment
    raw_down, down_val = extract_down_payment(text)
    result['down_payment_raw'] = raw_down
    result['down_payment'] = down_val

    # Residual
    raw_res, res_val = extract_residual_value(text)
    result['residual_value_raw'] = raw_res
    result['residual_value'] = res_val

    # Mileage & overage
    raw_mile, mile_val, raw_over, over_val = extract_mileage_and_overage(text)
    result['mileage_raw'] = raw_mile
    result['mileage_per_year'] = mile_val
    result['overage_raw'] = raw_over
    result['overage_per_mile'] = over_val

    # Buyout / purchase option
    raw_buy, buy_val = extract_buyout_price(text)
    result['buyout_raw'] = raw_buy
    result['buyout_price'] = buy_val

    # Clause snippets
    clauses = {}
    clauses['early_termination'] = extract_clause_snippet(text, ['early termination', 'termination', 'terminate'])
    clauses['maintenance'] = extract_clause_snippet(text, ['maintenance', 'service', 'maintain'])
    clauses['warranty'] = extract_clause_snippet(text, ['warranty', 'guarantee'])
    clauses['insurance'] = extract_clause_snippet(text, ['insurance', 'insurer', 'insured'])
    clauses['penalties'] = extract_clause_snippet(text, ['penalty', 'late fee', 'late payment', 'default'])

    # print result
    print("\n=== EXTRACTED (raw + normalized) ===\n")
    for k, v in result.items():
        print(f"{k:25} : {v}")
    print("\n=== CLAUSE SNIPPETS ===\n")
    for k, v in clauses.items():
        print(f"{k}:\n{v}\n")



=== EXTRACTED (raw + normalized) ===

apr_raw                   : 8.5%
apr_percent               : 8.5
lease_term_raw            : 36 months
lease_term_months         : 36
monthly_payment_raw       : 24,500 
monthly_payment           : 24500.0
down_payment_raw          : 1,50,000 
down_payment              : 150000.0
residual_value_raw        : 8,50,000 
residual_value            : 850000.0
mileage_raw               : None
mileage_per_year          : None
overage_raw               : 6
overage_per_mile          : 6.0
buyout_raw                : 7
buyout_price              : 7.0

=== CLAUSE SNIPPETS ===

early_termination:
er km Mileage is measured from lease start to lease end. 8. Early Termination Clause If the Lessee terminates the lease early: e Lessee must pay: o Remaining lease payments, 0R o Early termination fee of e 75,000, whichever is higher e Vehicle must be returned in good condition Early termination does not waive unpaid charges or damages. --- Page 3 --- 9. Pur

maintena

##vehicle details extraction

In [24]:
import requests

vin = "enter vin number"
url = f"https://vpic.nhtsa.dot.gov/api/vehicles/decodevinvalues/{vin}?format=json"

data = requests.get(url).json()
car = data["Results"][0]

for key, value in car.items():
    value = str(value).strip()
    if value:
        print(f"{key}: {value}")


AdditionalErrorText: Invalid character(s): 6:_, 8:I, 9:N, 10:_, 14:B, 15:E, 16:R.
ErrorCode: 6,7,11,400
ErrorText: 6 - Incomplete VIN; 7 - Manufacturer is not registered with NHTSA for sale or importation in the U.S. for use on U.S roads; Please contact the manufacturer directly for more information; 11 - Incorrect Model Year - Position 10 does not match valid model year codes (I, O, Q, U, Z, 0). Decoded data may not be accurate.; 400 - Invalid Characters Present
SuggestedVIN: ENTER!V!!!NUM!!!
VIN: enter vin number
VehicleDescriptor: ENTER VI* N


In [ ]:
import requests

vin = ""
url = f"https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{vin}?format=json"

response = requests.get(url)
data = response.json()

print("VIN:", vin)
print("SOURCE: NHTSA vPIC")
print("=" * 50)

for item in data["Results"]:
    variable = item["Variable"]
    value = item["Value"]
    if variable:
        print(f"{variable}: {value}")


##database setup+ integrations

In [26]:
import sqlite3

conn = sqlite3.connect("auto_platform.db")
cur = conn.cursor()

cur.execute("PRAGMA foreign_keys = ON;")


In [27]:
cur.executescript("""
CREATE TABLE users (
    id TEXT PRIMARY KEY,
    email TEXT NOT NULL,
    phone TEXT,
    full_name TEXT,
    auth_provider TEXT,
    created_at TEXT,
    updated_at TEXT
);

CREATE TABLE dealers (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    address_line1 TEXT,
    address_line2 TEXT,
    city TEXT,
    state TEXT,
    postal_code TEXT,
    country TEXT,
    phone TEXT,
    website TEXT,
    created_at TEXT
);

CREATE TABLE lenders (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    nmls_id TEXT,
    website TEXT,
    phone TEXT,
    address TEXT,
    created_at TEXT
);

CREATE TABLE vehicles (
    id TEXT PRIMARY KEY,
    vin TEXT NOT NULL,
    year INTEGER,
    make TEXT,
    model TEXT,
    trim TEXT,
    body_class TEXT,
    engine TEXT,
    drivetrain TEXT,
    fuel_type TEXT,
    odometer_miles REAL,
    color_ext TEXT,
    color_int TEXT,
    created_at TEXT,
    updated_at TEXT
);
""")


In [28]:
cur.executescript("""
CREATE TABLE providers (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    kind TEXT NOT NULL,
    is_free INTEGER NOT NULL,
    base_url TEXT,
    notes TEXT,
    created_at TEXT
);

CREATE TABLE vehicle_reports (
    id TEXT PRIMARY KEY,
    vehicle_id TEXT NOT NULL,
    provider_id TEXT,
    title TEXT,
    report_type TEXT,
    availability TEXT,
    url TEXT,
    raw TEXT,
    created_at TEXT,
    FOREIGN KEY(vehicle_id) REFERENCES vehicles(id),
    FOREIGN KEY(provider_id) REFERENCES providers(id)
);

CREATE TABLE vehicle_recalls (
    id TEXT PRIMARY KEY,
    vehicle_id TEXT NOT NULL,
    recall_number TEXT,
    issue_date TEXT,
    component TEXT,
    summary TEXT,
    remedy TEXT,
    source TEXT,
    raw TEXT,
    created_at TEXT,
    FOREIGN KEY(vehicle_id) REFERENCES vehicles(id)
);
""")


In [29]:
cur.executescript("""
CREATE TABLE contracts (
    id TEXT PRIMARY KEY,
    user_id TEXT,
    vehicle_id TEXT,
    dealer_id TEXT,
    lender_id TEXT,
    contract_type TEXT,
    doc_status TEXT,
    dealer_offer_name TEXT,
    contract_date TEXT,
    locale TEXT,
    currency TEXT,
    fairness_score REAL,
    red_flag_level TEXT,
    notes TEXT,
    created_at TEXT,
    updated_at TEXT,
    FOREIGN KEY(user_id) REFERENCES users(id),
    FOREIGN KEY(vehicle_id) REFERENCES vehicles(id),
    FOREIGN KEY(dealer_id) REFERENCES dealers(id),
    FOREIGN KEY(lender_id) REFERENCES lenders(id)
);

CREATE TABLE contract_files (
    id TEXT PRIMARY KEY,
    contract_id TEXT NOT NULL,
    storage_uri TEXT,
    file_name TEXT,
    mime_type TEXT,
    page_count INTEGER,
    uploaded_at TEXT,
    FOREIGN KEY(contract_id) REFERENCES contracts(id)
);

CREATE TABLE contract_pages (
    id TEXT PRIMARY KEY,
    contract_file_id TEXT NOT NULL,
    page_number INTEGER NOT NULL,
    ocr_text TEXT,
    ocr_confidence REAL,
    thumbnail_uri TEXT,
    created_at TEXT,
    FOREIGN KEY(contract_file_id) REFERENCES contract_files(id)
);
""")


In [30]:
cur.executescript("""
CREATE TABLE extractions (
    id TEXT PRIMARY KEY,
    contract_id TEXT NOT NULL,
    model_name TEXT,
    prompt_version TEXT,
    status TEXT,
    started_at TEXT,
    completed_at TEXT,
    raw_output TEXT,
    error_message TEXT,
    FOREIGN KEY(contract_id) REFERENCES contracts(id)
);

CREATE TABLE extracted_clauses (
    id TEXT PRIMARY KEY,
    extraction_id TEXT NOT NULL,
    clause_type TEXT,
    page_number INTEGER,
    text_snippet TEXT,
    normalized_value TEXT,
    red_flag_level TEXT,
    comment TEXT,
    FOREIGN KEY(extraction_id) REFERENCES extractions(id)
);

CREATE TABLE negotiation_threads (
    id TEXT PRIMARY KEY,
    user_id TEXT,
    contract_id TEXT,
    dealer_id TEXT,
    lender_id TEXT,
    channel TEXT,
    subject TEXT,
    created_at TEXT,
    closed_at TEXT
);

CREATE TABLE negotiation_messages (
    id TEXT PRIMARY KEY,
    thread_id TEXT NOT NULL,
    sender_role TEXT,
    body TEXT,
    suggested_text TEXT,
    attachments TEXT,
    sent_at TEXT,
    FOREIGN KEY(thread_id) REFERENCES negotiation_threads(id)
);
""")


In [31]:
cur.executescript("""
CREATE TABLE price_recommendations (
    id TEXT PRIMARY KEY,
    vehicle_id TEXT NOT NULL,
    geo_postal TEXT,
    msrp REAL,
    fair_price_low REAL,
    fair_price_high REAL,
    basis TEXT,
    methodology TEXT,
    generated_at TEXT,
    FOREIGN KEY(vehicle_id) REFERENCES vehicles(id)
);

CREATE TABLE offer_comparisons (
    id TEXT PRIMARY KEY,
    user_id TEXT NOT NULL,
    primary_contract_id TEXT NOT NULL,
    compared_contract_id TEXT NOT NULL,
    comparison_json TEXT,
    created_at TEXT
);

CREATE TABLE integration_logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    provider_id TEXT,
    request_path TEXT,
    request_params TEXT,
    response_status INTEGER,
    response_ms INTEGER,
    occurred_at TEXT,
    error_message TEXT
);
""")


In [32]:
cur.executescript("""
CREATE TABLE audit_events (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id TEXT,
    entity_table TEXT,
    entity_id TEXT,
    action TEXT,
    details TEXT,
    occurred_at TEXT
);
""")


In [33]:
conn.commit()

cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cur.fetchall())

conn.close()


[('users',), ('dealers',), ('lenders',), ('vehicles',), ('providers',), ('vehicle_reports',), ('vehicle_recalls',), ('contracts',), ('contract_files',), ('contract_pages',), ('extractions',), ('extracted_clauses',), ('negotiation_threads',), ('negotiation_messages',), ('price_recommendations',), ('offer_comparisons',), ('integration_logs',), ('sqlite_sequence',), ('audit_events',)]


In [ ]:
# Single script to insert the provided extraction, clauses and SLA-like fields
# into a local SQLite DB (auto_platform.db). Missing values are saved as NULL.
# IDs are simple strings (no UUID auto-generation).

import sqlite3
import json
from datetime import datetime, timezone

DB_PATH = "auto_platform.db"

def now_utc_iso():
    return datetime.now(timezone.utc).isoformat()

# Ensure minimal tables exist (safe to run even if you already created the full schema)
def ensure_tables():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    cur.executescript("""
    PRAGMA foreign_keys = ON;

    CREATE TABLE IF NOT EXISTS extractions (
        id TEXT PRIMARY KEY,
        contract_id TEXT,
        model_name TEXT,
        prompt_version TEXT,
        status TEXT,
        started_at TEXT,
        completed_at TEXT,
        raw_output TEXT,
        error_message TEXT
    );

    CREATE TABLE IF NOT EXISTS extracted_clauses (
        id TEXT PRIMARY KEY,
        extraction_id TEXT,
        clause_type TEXT,
        page_number INTEGER,
        text_snippet TEXT,
        normalized_value TEXT,
        red_flag_level TEXT,
        comment TEXT
    );

    CREATE TABLE IF NOT EXISTS contract_sla (
        id TEXT PRIMARY KEY,
        contract_id TEXT NOT NULL,
        apr_percent REAL,
        money_factor REAL,
        term_months INTEGER,
        monthly_payment REAL,
        down_payment REAL,
        fees_total REAL,
        residual_value REAL,
        residual_percent_msrp REAL,
        msrp REAL,
        cap_cost REAL,
        cap_cost_reduction REAL,
        mileage_allowance_yr INTEGER,
        mileage_overage_fee REAL,
        early_termination_fee REAL,
        purchase_option_price REAL,
        buyout_price TEXT,
        other_terms TEXT,
        created_at TEXT,
        updated_at TEXT
    );
    """)
    conn.commit()
    conn.close()

def insert_extraction(extraction):
    """
    extraction: dict with keys matching columns of extractions table.
    If id not present a simple id is created (extraction_1 style) but by default we prefer passed id.
    """
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    keys = []
    vals = []
    for k, v in extraction.items():
        keys.append(k)
        if isinstance(v, (dict, list)):
            vals.append(json.dumps(v))
        else:
            vals.append(v)
    placeholders = ",".join(["?"] * len(keys))
    cols = ",".join(keys)

    cur.execute(f"INSERT OR REPLACE INTO extractions ({cols}) VALUES ({placeholders})", tuple(vals))
    conn.commit()
    conn.close()

def insert_extracted_clause(clause):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    columns = [
        "id",
        "extraction_id",
        "clause_type",
        "page_number",
        "text_snippet",
        "normalized_value",
        "red_flag_level",
        "comment"
    ]

    vals = []
    for c in columns:
        v = clause.get(c)
        if isinstance(v, (dict, list)):
            v = json.dumps(v)
        vals.append(v)

    placeholders = ",".join(["?"] * len(columns))
    cols = ",".join(columns)

    cur.execute(f"INSERT OR REPLACE INTO extracted_clauses ({cols}) VALUES ({placeholders})", tuple(vals))
    conn.commit()
    conn.close()

def insert_contract_sla(sla):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Define allowed columns (matching table above)
    columns = [
        "id", "contract_id", "apr_percent", "money_factor", "term_months",
        "monthly_payment", "down_payment", "fees_total", "residual_value",
        "residual_percent_msrp", "msrp", "cap_cost", "cap_cost_reduction",
        "mileage_allowance_yr", "mileage_overage_fee", "early_termination_fee",
        "purchase_option_price", "buyout_price", "other_terms", "created_at", "updated_at"
    ]

    vals = []
    for c in columns:
        v = sla.get(c)
        # serialize complex structures
        if isinstance(v, (dict, list, tuple)):
            v = json.dumps(v)
        vals.append(v)

    placeholders = ",".join(["?"] * len(columns))
    cols = ",".join(columns)

    cur.execute(f"INSERT OR REPLACE INTO contract_sla ({cols}) VALUES ({placeholders})", tuple(vals))
    conn.commit()
    conn.close()

def fetch_all(table):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM {table}")
    rows = cur.fetchall()
    conn.close()
    return rows

# ------------------------
# Now prepare the provided extracted data
# ------------------------
ensure_tables()

# Use simple IDs for now (you can replace these with your own IDs later)
EXTRACTION_ID = "extraction_001"
CONTRACT_ID = "contract_001"
SLA_ID = "sla_001"

# Raw+normalized block (as provided by you)
extraction_record = {
    "id": EXTRACTION_ID,
    "contract_id": CONTRACT_ID,
    "model_name": "llm-extractor-v1",
    "prompt_version": "v1",
    "status": "completed",
    "started_at": now_utc_iso(),
    "completed_at": now_utc_iso(),
    "raw_output": json.dumps({
        "apr_raw": "8.5%",
        "apr_percent": 8.5,
        "lease_term_raw": "36 months",
        "lease_term_months": 36,
        "monthly_payment_raw": "42,500",
        "monthly_payment": 42500.0,
        "down_payment_raw": "150,000",
        "down_payment": 150000.0,
        "residual_value_raw": "900,000",
        "residual_value": 900000.0,
        "mileage_raw": "15,000",
        "mileage_per_year": 15000,
        "overage_raw": "312",
        "overage_per_mile": 312.0,
        "buyout_raw": None,
        "buyout_price": [None, None]
    }, ensure_ascii=False)
}

insert_extraction(extraction_record)

# Insert contract_sla row using normalized numeric fields where available.
sla_record = {
    "id": SLA_ID,
    "contract_id": CONTRACT_ID,
    "apr_percent": extraction_record and json.loads(extraction_record["raw_output"]).get("apr_percent"),
    "term_months": json.loads(extraction_record["raw_output"]).get("lease_term_months"),
    "monthly_payment": json.loads(extraction_record["raw_output"]).get("monthly_payment"),
    "down_payment": json.loads(extraction_record["raw_output"]).get("down_payment"),
    "residual_value": json.loads(extraction_record["raw_output"]).get("residual_value"),
    "mileage_allowance_yr": json.loads(extraction_record["raw_output"]).get("mileage_per_year"),
    "mileage_overage_fee": json.loads(extraction_record["raw_output"]).get("overage_per_mile"),
    # buyout_price is a tuple/list in the raw; store as JSON string (or NULL if all None)
    "buyout_price": json.loads(extraction_record["raw_output"]).get("buyout_price"),
    "created_at": now_utc_iso(),
    "updated_at": now_utc_iso()
}
insert_contract_sla(sla_record)

# Clause snippets (from your CLAUSE SNIPPETS). We'll store the snippet text and any small structured normalizations.
clauses = [
    {
        "id": "clause_early_termination",
        "extraction_id": EXTRACTION_ID,
        "clause_type": "early_termination",
        "page_number": 2,
        "text_snippet": ("Early termination of this lease before the Lease End Date will result in: "
                         "a) Payment of all outstanding lease payments b) An early termination fee of 75,000 "
                         "c) Any applicable excess mileage or damage charges 9. PURCHASE OPTION ..."),
        "normalized_value": {
            "early_termination_fee": 75000.0,
            "actions": ["pay outstanding payments", "pay early termination fee", "pay excess mileage/damage charges"]
        },
        "red_flag_level": "high",
        "comment": None
    },
    {
        "id": "clause_maintenance",
        "extraction_id": EXTRACTION_ID,
        "clause_type": "maintenance",
        "page_number": 2,
        "text_snippet": ("The Lessee is responsible for routine maintenance including oil changes, tire replacement, "
                         "brake servicing, and general wear and tear. Major mechanical repairs covered under manufacturer warranty."),
        "normalized_value": None,
        "red_flag_level": None,
        "comment": None
    },
    {
        "id": "clause_warranty",
        "extraction_id": EXTRACTION_ID,
        "clause_type": "warranty",
        "page_number": 2,
        "text_snippet": ("The vehicle is covered under the manufacturer’s standard warranty for 3 years or 100,000 km, whichever occurs first."),
        "normalized_value": {"warranty_years": 3, "warranty_km": 100000},
        "red_flag_level": None,
        "comment": None
    },
    {
        "id": "clause_insurance",
        "extraction_id": EXTRACTION_ID,
        "clause_type": "insurance",
        "page_number": 2,
        "text_snippet": ("Lessee must maintain comprehensive motor insurance including third-party liability coverage throughout the lease term."),
        "normalized_value": None,
        "red_flag_level": None,
        "comment": None
    },
    {
        "id": "clause_penalties",
        "extraction_id": EXTRACTION_ID,
        "clause_type": "penalties",
        "page_number": 3,
        "text_snippet": ("Late Payment Fee: 31,500 per occurrence. Interest on overdue amounts: 2% per month. "
                         "Vehicle use restrictions: no commercial transport, racing, or illegal activities."),
        "normalized_value": {"late_fee_per_occurrence": 31500.0, "late_interest_monthly_pct": 2.0},
        "red_flag_level": None,
        "comment": None
    }
]

for c in clauses:
    insert_extracted_clause(c)

# ------------------------
# Verification: print what we inserted
# ------------------------
print("=== extractions ===")
for r in fetch_all("extractions"):
    print(r)

print("\n=== contract_sla ===")
for r in fetch_all("contract_sla"):
    print(r)

print("\n=== extracted_clauses ===")
for r in fetch_all("extracted_clauses"):
    # attempt to pretty-print normalized_value if present
    row = list(r)
    nv = row[5]  # normalized_value column
    try:
        row[5] = json.loads(nv) if nv else None
    except Exception:
        row[5] = nv
    print(row)


=== extractions ===
('extraction_001', 'contract_001', 'llm-extractor-v1', 'v1', 'completed', '2026-01-22T13:11:17.747182+00:00', '2026-01-22T13:11:17.747208+00:00', '{"apr_raw": "8.5%", "apr_percent": 8.5, "lease_term_raw": "36 months", "lease_term_months": 36, "monthly_payment_raw": "42,500", "monthly_payment": 42500.0, "down_payment_raw": "150,000", "down_payment": 150000.0, "residual_value_raw": "900,000", "residual_value": 900000.0, "mileage_raw": "15,000", "mileage_per_year": 15000, "overage_raw": "312", "overage_per_mile": 312.0, "buyout_raw": null, "buyout_price": [null, null]}', None)

=== contract_sla ===
('sla_001', 'contract_001', 8.5, None, 36, 42500.0, 150000.0, None, 900000.0, None, None, None, None, 15000, 312.0, None, None, '[null, null]', None, '2026-01-22T13:11:17.755280+00:00', '2026-01-22T13:11:17.755300+00:00')

=== extracted_clauses ===
['clause_early_termination', 'extraction_001', 'early_termination', 2, 'Early termination of this lease before the Lease End Dat